In [12]:
import face_recognition
import cv2
import threading
import queue

In [13]:
# Example usage
image1 = cv2.imread("keanu.jpeg")
image2 = cv2.imread("murphy.jpeg")
face_location_image1 = face_recognition.face_locations(image1, model="hog")
face_location_image2 = face_recognition.face_locations(image2, model="hog")

known_faces = [
    (face_recognition.face_encodings(image1, face_location_image1)[0], "keanu"),
    (face_recognition.face_encodings(image2, face_location_image2)[0], "murphy"),
]

In [14]:
known_face_encodings, known_face_names = zip(*known_faces)
# face_match = False
# # obs_frames = queue.Queue()
# obs_frames = []

In [15]:
def recognize_faces(image):
    # global obs_frame, face_match
    image = cv2.resize(image, (640, 480))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(image_rgb, model="hog")
    face_encodings = face_recognition.face_encodings(image_rgb, face_locations)
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
        name = "Person"
        if True in matches:
            match_index = matches.index(True)
            name = known_face_names[match_index]
        top, right, bottom, left = face_location
        cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(image, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
    cv2.imshow("Frame", image)

In [16]:
image = None

In [17]:
image

In [18]:
camera = cv2.VideoCapture(0)
frame_count = 0

while True:
    ret, frame = camera.read()

    if ret:
        if frame_count % 10 == 0:
            threading.Thread(target=recognize_faces, args=(frame,)).start()
            frame_count = 0
        
        if face_match:
            try:
                cv2.imshow("Camera Feed", obs_frames.pop(0))
                # obs_frames.task_done()
            except:
                cv2.imshow("Camera Feed", frame)
        
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        print("Camera not working")
        break
    frame_count += 1

camera.release()
cv2.destroyAllWindows()

NameError: name 'face_match' is not defined

In [2]:
from mtcnn.mtcnn import MTCNN
import cv2


In [3]:
detector = MTCNN()

In [19]:


def detect_faces(image):
    # Load the input image
    # image = cv2.imread(image_path)
    
    # Convert the image to RGB (face_recognition works with RGB images)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Find all face locations in the image
    face_locations = face_recognition.face_locations(image_rgb, model="hog")
    
    # Draw rectangles around the detected faces
    for face_location in face_locations:
        top, right, bottom, left = face_location
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    
    # Show the image with detected faces
    cv2.imshow("Detected Faces", cv2.resize(image, (800, 600)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [20]:
# Example usage
try:
    image_path = "vishal.jpg"
    image = cv2.imread(image_path)
    detect_faces(image)
except Exception as e:
    print(e)
    print("No faces were found.")


In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf


In [2]:

# Constants
train_data_dir = 'C:/Users/saivishal radham/Desktop/Stuff/python/train_images/'  # Directory containing the labeled face images
# validation_data_dir = 'path_to_validation_data'  # Directory containing the validation data
new_face_dir = 'C:/Users/saivishal radham/Desktop/Stuff/python/new_face_dir/'  # Directory containing the new face image
num_epochs = 20
batch_size = 32

class_labels = os.listdir(train_data_dir)
num_classes = len(class_labels)


In [16]:

if os.path.exists('face_recognition_model.h5'):
    model = tf.keras.models.load_model('face_recognition_model.h5')

else:
    # Load VGG16 model without the top classification layers
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    # Freeze the base model layers to prevent them from being retrained
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers for face recognition
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)  # Adjust the size of the dense layer as per your requirements
    output = Dense(num_classes, activation='softmax')(x)  # num_classes = number of faces to be recognized

    # Create the final model
    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\saivishal radham\Desktop\Stuff\python\facerecog\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4 images belonging to 4 classes.


In [6]:

# Define a learning rate scheduler to reduce the learning rate during fine-tuning
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 5 == 0:
        return lr * 0.1
    return lr


In [13]:
! pip install sklearn

  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2970 sha256=eaca6bc7f3185b0358c48178ed192c45b1249406c7715891c04347572446df7d
  Stored in directory: c:\users\saivishal radham\appdata\local\pip\cache\wheels\f3\6d\ba\d999da6dff3d5cc6735e9855579fa236bf7c5bdbb210a873b8
Successfully built sklearn


In [19]:
import scipy as scipy

In [23]:

# Fine-tune the model
# model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs)

# Save the updated model
model.save('face_recognition_model.h5')


NameError: name 'scipy' is not defined

In [24]:
# Perform fine-tuning
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Print the class indices for debugging purposes
print("Class Indices:", train_generator.class_indices)

# Check the shape of the images in the generator for debugging purposes
for images, labels in train_generator:
    print("Batch Shape:", images.shape)
    print("Label Shape:", labels.shape)
    break

# Define a learning rate scheduler to reduce the learning rate during fine-tuning
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 5 == 0:
        return lr * 0.1
    return lr

# Fine-tune the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, callbacks=[LearningRateScheduler(lr_scheduler)])


Found 4 images belonging to 4 classes.
Class Indices: {'keanu': 0, 'pavan': 1, 'vinay': 2, 'vishal': 3}


NameError: name 'scipy' is not defined